In [ ]:
# This file is used to test how to get data from the Frax API and supplied info.
# We may also sanity check or use chain access via a local node or blockchain scanner
# if that's necessary

In [ ]:
from cache import cache
from collections import defaultdict
from decimal import Decimal
import httpx
import json
from pprint import pprint
import sys
from tabulate import tabulate
from web3 import Web3

In [ ]:
# These are all the variables that we'll load either from our cache or override locally:w

# We're going to need this to get ABIs easily
ETHERSCAN_KEY = cache.get_simple('ETHERSCAN_KEY')
if not ETHERSCAN_KEY:
  ETHERSCAN_KEY = "YourApiKeyToken"

# Replace this with the address you want to see...
ADDRESS = cache.get_simple('ADDRESS')
if not ADDRESS:
  # We'll default to fraximalist.eth as a default example, a big whale with a bunch of staked gauges
  ADDRESS = '0x68e912af6176bb1639aca936eb58d71cc2558b66'
ADDRESS = Web3.toChecksumAddress(ADDRESS)

# You are going to need an Ethereum RPC to talk to...
RPC_URL = 'http://localhost:8545/'

# You shouldn't usually need this since the cache is mainly for ABIs and Frax Pool List, not balances 
REFRESH = False

In [ ]:
# Get Gauge Pools and sort by gauge_index
req = cache.get('https://api.frax.finance/gauge/info', refresh=REFRESH)
gauges = json.loads(req['value'])
'''
# A gauge looks like:

{'address': '0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0',
 'fxs_next': 39981.83490806742,
 'fxs_now': 33694.5538302253,
 'gauge_index': 0,
 'gauge_weight': '17127149135922894237322600',
 'name': 'Uniswap V3 FRAX/USDC',
 'relative_weight_next_pct': 45.69352560921991,
 'relative_weight_next_raw': '456935256092199104',
 'relative_weight_now_pct': 38.508061520257485,
 'relative_weight_now_raw': '385080615202574845'}
'''
gauges.sort(key=lambda g: g['gauge_index'])
for gauge in gauges:
  print(gauge['name'])
  
# We can also get some of this information from easily from the blockchain
# Note for each chain you'd need to query a node (and potentially a blockchain scanner for ABI)

def get_abi(contract):
  URL = f'https://api.etherscan.io/api?module=contract&action=getabi&address={contract}&apikey={ETHERSCAN_KEY}'
  req = cache.get(URL, refresh=REFRESH)
  r = json.loads(req['value'])
  return r['result']

def get_contract_details(contract):
  URL = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract}&apikey={ETHERSCAN_KEY}'
  req = cache.get(URL, refresh=REFRESH)
  r = json.loads(req['value'])
  return r['result'][0]

# Use a local node...
w3 = Web3(Web3.HTTPProvider(RPC_URL))

GAUGE = '0x3669c421b77340b2979d1a00a792cc2ee0fce737'
GAUGE = Web3.toChecksumAddress(GAUGE)
c_gauge = w3.eth.contract(address=GAUGE, abi=get_abi(GAUGE))

farms = []
n = c_gauge.functions.n_gauges().call()
for i in range(n):
  r = c_gauge.functions.gauges(i).call()
  farms.append(r)

# Get farm details
print()
for farm in farms:
  details = get_contract_details(farm)
  print(details['ContractName'])
  print(farm)

  c_farm = w3.eth.contract(address=farm, abi=details['ABI'])

  try:
    reward_tokens = c_farm.functions.getAllRewardTokens().call()
    print('Reward Tokens:')
    pprint(reward_tokens)
  except:
    print('No getAllRewardTokens() - FXS? 0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0')
    pass
  print()

In [ ]:
# Get all pools listed in the "Staking" tab
req = cache.get('https://api.frax.finance/pools', refresh=REFRESH)
pools = json.loads(req['value'])
'''
# A pool looks like
{'apy': 18.786202278137203,
  'apy_max': 56.35860683441161,
  'chain': 'arbitrum',
  'identifier': 'Curve VSTFRAX-f',
  'is_deprecated': False,
  'liquidity_locked': 24336315.042461667,
  'logo': '',
  'pair': 'Curve VSTFRAX-f',
  'pairLink': 'https://app.frax.finance/staking#Curve_VSTFRAX_F',
  'platform': 'curve_arbi_vstfrax',
  'pool_rewards': ['FXS', 'VSTA'],
  'pool_tokens': ['FRAX', 'VST']},
'''

# Huge PITA but we can get contract addresses for all these pools as well...
req = cache.get('https://raw.githubusercontent.com/FraxFinance/frax-solidity/master/src/types/constants.ts', refresh=REFRESH)
constants = req['value'].decode('utf8')
start = constants.find('export const CONTRACT_ADDRESSES')
end = constants.find('export const INVESTOR_ALLOCATIONS')
constants = constants[start:end].replace('export const CONTRACT_ADDRESSES = ', '')

# best tool: https://regex101.com/
# also useful: https://pythex.org/
# docs: https://note.nkmk.me/en/python-str-replace-translate-re-sub/
# https://jsonlint.com/
import re
constants = re.sub(r'//.*', '', constants)
constants = re.sub(r'[\s]*NOTE.*', '', constants)
constants = re.sub(r'([a-zA-Z0-9_]+)[\s]*:\B', r'"\1":', constants)
import ast
contracts = ast.literal_eval(constants)

staking_contracts = {}
for key in contracts:
  if 'staking_contracts' in contracts[key]:
    for contract in contracts[key]['staking_contracts']:
      staking_contracts[contract] = {'chain': key, 'address': contracts[key]['staking_contracts'][contract]}
    
data = []
for gauge in gauges:
  try:
    contract = staking_contracts[gauge['name']]
    data.append((contract['chain'], gauge['name'], contract['address']))
  except:
    print(f'NO CONTRACT for {gauge["name"]}')

print(tabulate(data, headers=['chain', 'gauge', 'address']))


In [ ]:
# let's gather useful pool data...
for pool in pools:
  name = pool['identifier']
  if name in staking_contracts:
    staking_contracts[name]['pool'] = pool

# How to get staked info from Frax API for an address
def get_user_stake_info(address, chain, gauge_id):
  URL = 'https://api.frax.finance/stakedata/user-stake-info'
  data = {'the_chain': chain,
          'staking_choice': gauge_id,
          'staker_address': ADDRESS}
  req = httpx.post(URL, data=data)
  return req.content

for gauge in gauges:
  name = gauge['name']
  contract = staking_contracts[name]
  apy = contract['pool']['apy']
  gauge['apy'] = apy
  apy_max = contract['pool']['apy_max']
  gauge['apy_max'] = apy_max
  chain = contract['chain']
  gauge['chain'] = chain

  content = get_user_stake_info(ADDRESS, chain, name)
  info = json.loads(content)
  contract['info'] = info
  gauge['info'] = info
  # pprint(info)
  total_value = info['locked_total_usd_value'] + info['unlocked_total_usd_value'] + info['lp_token_bal_usd']
  gauge['user_total_value'] = total_value
  print()
  print(f"{contract['chain']:10} : {gauge['name']:24} ({apy:7.2f}/{apy_max:7.2f})% : ${total_value:12,.2f}")
  for reward in info['rewards']:
    print(f"  {reward['reward_balance']:12,.3f} {reward['token']:6} (${reward['reward_usd_value']:,.2f})")

  # We need to get our veFXS Multiplier onchain?
  c_address = Web3.toChecksumAddress(gauge['address'])
  c_gauge = w3.eth.contract(address=c_address, abi=get_abi(c_address))
  try:
    gauge['vefxs_multiplier'] = c_gauge.functions.veFXSMultiplier(ADDRESS).call()
    print(f"veFXS Multiplier: {gauge['vefxs_multiplier']/1000000000000000000:0.2f}")
  except:
    # Middleman gauges
    gauge['vefxs_multiplier'] = 0
    print("Couldn't get veFXS Multiplier, setting to 0")

  
# We can also look onchain
'''
ABI notes...

FraxFarm_UniV3_veFXS_FRAX_USDC
is FraxFarm_UniV3_veFXS
https://etherscan.io/address/0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0#readContract
earned(address)
userStakedFrax(address)
uni_token0
uni_token1
combinedWeightOf(address)
minVeFXSForMaxBoost(address)

FraxMiddlemanGauge_FRAX_mUSD
is FraxMiddlemanGauge
https://etherscan.io/address/0x3e14f6EEDCC5Bc1d0Fc7B20B45eAE7B1F74a6AeC#readContract
name
bridge_address
destination_address_override
-> you need to find the chain yourself...
https://polygonscan.com/address/0xc425Fd9Ed3C892d849C9E1a971516da1C1B29696#readContract
FraxCrossChainFarm_FRAX_mUSD
is FraxCrossChainFarm
combinedWeightOf()
earned()
lockedLiquidityOf(address)
lockedStakesOf(address)
rewardsToken0
rewardsToken1
userRewardPerTokenPaid0(address)
userRewardPerTokenPaid1(address)
stakingToken

FraxUniV3Farm_Stable_FRAX_DAI 
is FraxUniV3Farm_Stable
earned(address)
userStakedFrax(address)
uni_token0
uni_token1
combinedWeightOf(address)
minVeFXSForMaxBoost(address)

StakingRewardsMultiGauge_FRAX_SUSHI
is StakingRewardsMultiGauge
https://etherscan.io/address/0xb4Ab0dE6581FBD3A02cF8f9f265138691c3A7d5D#readContract
earned(address)
getAllRewardTokens
try:
  user_frax = c_farm.functions.userStakedFrax(user).call()
      user_frax = Web3.fromWei(user_frax, 'ether')
      print(f'FRAX balance: ${user_frax:,.2f}')
    except:
      pass
'''
print()

In [ ]:
# Let's get veFXS Data 
URL = 'https://api.frax.finance/vefxsdata/1-hour'
req = cache.get(URL, refresh=True)
r = json.loads(req['value'])
max_apr = r['full_items'][-1]['apr']

# We need the latest coin data to calculate pricing
URL = 'https://api.frax.finance/priceitems/coin-stats'
req = cache.get(URL, refresh=True)
r = json.loads(req['value'])

# Get veFXS multiplier, amounts
VEFXS_CONTRACT = '0xc8418aF6358FFddA74e09Ca9CC3Fe03Ca6aDC5b0'
details = get_contract_details(VEFXS_CONTRACT)
c_vefxs = w3.eth.contract(address=VEFXS_CONTRACT, abi=details['ABI'])
slope = c_vefxs.functions.get_last_user_slope(ADDRESS).call()
slope = slope/10**15
user_apr = max_apr*slope

balance = c_vefxs.functions.balanceOf(ADDRESS).call()

amount, ending_timestamp = c_vefxs.functions.locked(ADDRESS).call()
amount = amount/10**18

fxs_price = r['FXS']['price']
dollar_value = amount * fxs_price

# Get Rewards to collected
YIELD_CONTRACT = '0xc6764e58b36e26b08Fd1d2AeD4538c02171fA872'  # veFXSYieldDistributorV4
details = get_contract_details(YIELD_CONTRACT)
c_yield = w3.eth.contract(address=YIELD_CONTRACT, abi=details['ABI'])
yield_earned = c_yield.functions.earned(ADDRESS).call()
yield_earned = yield_earned/10**18

0x3f3b9b0f14def5817a61437ccb25ccbbd4853ffd
gauge = {
    'address': VEFXS_CONTRACT,
    'apy': user_apr,
    'apy_max': max_apr,
    'chain': 'ethereum',
    'info': {
        'locked_stakes': [{
            'dollar_value': dollar_value,
            'ending_timestamp': ending_timestamp,
            'lock_multiplier': 0.0,
        }],
        'locked_total': amount,
        'locked_total_usd_value': dollar_value,
        'rewards': [{'chain': 'ethereum',
                       'coingecko_slug': 'frax-share',
                       'price': fxs_price,
                       'reward_balance': yield_earned,
                       'reward_rate': '65317493739689486',
                       'reward_usd_value': yield_earned*fxs_price,
                       'token': 'FXS'
                    }],
        'rewards_total_usd_value': yield_earned*fxs_price,
    },
    'name': 'veFXS',
    'user_total_value': dollar_value,
    'vefxs_multiplier': 1000000000000000000,
}
    
gauges.insert(0, gauge)

In [ ]:
# Only for modifying gauges
# del(gauges[1])

In [ ]:
# I've been told the "Uniswap V3 FRAX/agEUR" gauge is wrong?
for gauge in gauges:
    if gauge['name'] == 'Uniswap V3 FRAX/agEUR':
        usd_total_value = 0.0
        for stake in gauge['info']['locked_stakes']:
             # Turns out that calculating a UniV3 position is hideously complex. Instead let's cheat.
            APYVISION_URL = f'https://stats.apy.vision/api/v1/uniswapv3/uniswapv3_eth/positions/{stake["token_id"]}'
            r = cache.get(APYVISION_URL)
            r = json.loads(r['value'])
            stake['dollar_value'] =  r['day_datas'][0]['position_usd_value_at_block']
            usd_total_value += stake['dollar_value']
            
        gauge['user_total_value'] = usd_total_value

In [ ]:
# Total in Pools
total_usd = 0.0
for gauge in gauges:
    total_usd += gauge['user_total_value']
    
data = []
total_daily = 0.0
for gauge in gauges:
    # Skip empty pools
    if not gauge['user_total_value']:
        continue
    
    # Pools
    row = []
    row.append(gauge['chain'])
    row.append(gauge['name'])
    apy = f"({gauge['apy']:.2f}%/{gauge['apy_max']:.2f}%)"
    row.append(apy)
    row.append(f"${gauge['user_total_value']:,.2f}")
    row.append(f"{100*gauge['user_total_value']/total_usd:,.1f}%")
    data.append(row)

    
    # Let's calculate our personal APR
    vefxs_multiplier = float(gauge['vefxs_multiplier'])/10**18
    total_stake_multiplier = 0.0
    for stake in gauge['info']['locked_stakes']:
        ### if we want to print out unlock times...
        # stake['kek_id']
        # stake['ending_timestamp']
        # stake['start_timestamp']
        total_stake_percent = stake['dollar_value']/gauge['user_total_value']
        stake_multiplier = int(stake['lock_multiplier'])/10**18
        
        total_stake_multiplier += total_stake_percent * stake_multiplier
            
    user_apr = gauge['apy']*(vefxs_multiplier+total_stake_multiplier)
    
    ## Get earnings...
    earnings = ''
    for reward in gauge['info']['rewards']:
        earnings += f".  {reward['reward_balance']:12,.3f} {reward['token']:6} (${reward['reward_usd_value']:,.2f})\n"
    
    row = []
    row.append('')
    row.append(earnings)
    row.append(f'{user_apr:.2f}%')
    daily = user_apr/100/365*gauge['user_total_value']
    total_daily += daily
    row.append(f'${daily:,.2f}/d')
    data.append(row)
    
    # empty
    data.append(('-'))
              

headers = ('chain', 'pool', 'apr/max apr', 'amount', '%')
colalign = ('left', 'left', 'right', 'right', 'right')
print(tabulate(data, headers=headers, colalign=colalign))
print()
print(f'Daily Yield     : ${total_daily:,.2f}')
print(f'Total Deposited : ${total_usd:,.2f}')
print(f'Average APR     : {100*365*total_daily/total_usd:,.2f}%')